In [ ]:
import os
from dotenv import load_dotenv
from pydantic import BaseModel
import google.genai as genai
from google.genai import types
from langchain.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.language_models import BaseChatModel
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableLambda,RunnableMap
from langchain_core.messages import AIMessage
from langchain_core.prompt_values import StringPromptValue
import gradio as gr

In [ ]:
load_dotenv(override=True)

api_key = os.getenv("GEMINI_API_KEY")

In [ ]:
loader = PyPDFLoader('./assets/constitution.pdf')
pdf = loader.load()


In [ ]:
model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vector_store = FAISS.from_documents(embedding=model,documents=pdf)

In [ ]:
class KeyWords(BaseModel):
    keywords: list[str]
    
client = genai.Client(api_key=api_key)
    
sys = types.GenerateContentConfig(
    temperature=0.2,
    response_schema=KeyWords,
    response_mime_type="application/json",
    system_instruction="On the basis of the user's question, return a list of at least 5 relevant keywords (terms, article numbers, or important phrases) which can be used for searching within the Indian Constitution."
)
def retriever(query,api_key=api_key):
    response = client.models.generate_content(
        contents = query["question"],
        model="gemini-2.0-flash",
        config= sys
    ).parsed
    if(len(response.keywords) == 0):
        return "There is no relevent context in the document, answer generically if possible"
    search = " ".join(response.keywords)
    content = vector_store.similarity_search(search,5)
    context = """"""
    for page in content:
        context += page.page_content + "\n"
    return context

In [ ]:
class customModel(BaseChatModel):
    def __init__(self,api_key:str):
        client = genai.Client(api_key=api_key)
        object.__setattr__(self,"client",client)
        system = types.GenerateContentConfig(
            temperature=0.8,
            system_instruction="""You are a chatbot that will answer the questions Related to Indian Constitution.
            You will be given context, which are a few pages of indian constitution and the question to answer the answer of questions related to Indian constitution.
            You can answer generic question by urself and also you can use your info in answering questions
            """
        )
        object.__setattr__(self,"system",system)
        
    def _generate(self, messages:str)->str:
        response = self.client.models.generate_content(
            contents=messages,
            model="gemini-2.0-flash",
            config=self.system
        )
        return response.text
    
    def invoke(self, input:StringPromptValue, config = None)->AIMessage:
        question = input.to_string()
        response = self._generate(question)
        return AIMessage(content=response)
         
    @property
    def _llm_type(self):
        return "custom_gemini"
    
    @property
    def callback(self):
        return []
    
    @property
    def streaming(self):
        return False

In [ ]:
prompt_template = ChatPromptTemplate.from_template("""
Answer the following questions on the basis of given context.
question : {question}
context : {context}                                                   
""")

llm = customModel(api_key=api_key)

In [ ]:
chain = (RunnableMap({"context":RunnableLambda(retriever),"question":RunnablePassthrough()})
| prompt_template
| llm
| StrOutputParser()         
)

In [ ]:
def chat(messages,history):
    return chain.invoke({"question":messages})

In [ ]:
gr.ChatInterface(chat).launch()